<a href="https://colab.research.google.com/github/karl-gardner/dropletdetection/blob/main/yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a align="left" href="https://ultralytics.com/yolov5" target="_blank">
<img width="1024", src="https://user-images.githubusercontent.com/26833433/125273437-35b3fc00-e30d-11eb-9079-46f313325424.png"></a>

This is the **official YOLOv5 🚀 notebook** by **Ultralytics**, and is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/). 
For more information please visit https://github.com/ultralytics/yolov5 and https://ultralytics.com. Thank you!

# 0. Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla P100-PCIE-16GB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Curl droplet dataset from roboflow (PC3DropletDetection2)

Data With Augmentation for Training (final_dataset)

In [ ]:
%cd /content
!curl -L "[ROBOFLOW-API-KEY]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
clear_output()

Data with No Augmentation (No_Augmentation)

In [ ]:
# %cd /content
# !curl -L "[ROBOFLOW-API-KEY]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# clear_output()

# 2. Train

<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="1000" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615338ba77195c71bd2c5ab1_computer-vision-flow.png"/></a></p>
Close the active learning loop by sampling images from your inference conditions with the `roboflow` pip package
<br><br>

Train a YOLOv5s model on the [COCO128](https://www.kaggle.com/ultralytics/coco128) dataset with `--data coco128.yaml`, starting from pretrained `--weights yolov5s.pt`, or from randomly initialized `--weights '' --cfg yolov5s.yaml`.

- **Pretrained [Models](https://github.com/ultralytics/yolov5/tree/master/models)** are downloaded
automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases)
- **[Datasets](https://github.com/ultralytics/yolov5/tree/master/data)** available for autodownload include: [COCO](https://github.com/ultralytics/yolov5/blob/master/data/coco.yaml), [COCO128](https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml), [VOC](https://github.com/ultralytics/yolov5/blob/master/data/VOC.yaml), [Argoverse](https://github.com/ultralytics/yolov5/blob/master/data/Argoverse.yaml), [VisDrone](https://github.com/ultralytics/yolov5/blob/master/data/VisDrone.yaml), [GlobalWheat](https://github.com/ultralytics/yolov5/blob/master/data/GlobalWheat2020.yaml), [xView](https://github.com/ultralytics/yolov5/blob/master/data/xView.yaml), [Objects365](https://github.com/ultralytics/yolov5/blob/master/data/Objects365.yaml), [SKU-110K](https://github.com/ultralytics/yolov5/blob/master/data/SKU-110K.yaml).
- **Training Results** are saved to `runs/train/` with incrementing run directories, i.e. `runs/train/exp2`, `runs/train/exp3` etc.
<br><br>

## Train on Custom Data with Roboflow 🌟 NEW

[Roboflow](https://roboflow.com/?ref=ultralytics) enables you to easily **organize, label, and prepare** a high quality dataset with your own custom data. Roboflow also makes it easy to establish an active learning pipeline, collaborate with your team on dataset improvement, and integrate directly into your model building workflow with the `roboflow` pip package.

- Custom Training Example: [https://blog.roboflow.com/how-to-train-yolov5-on-a-custom-dataset/](https://blog.roboflow.com/how-to-train-yolov5-on-a-custom-dataset/?ref=ultralytics)
- Custom Training Notebook: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/yolov5-custom-training-tutorial/blob/main/yolov5-custom-training.ipynb)
<br>

<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="480" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/6152a275ad4b4ac20cd2e21a_roboflow-annotate.gif"/></a></p>Label images lightning fast (including with model-assisted labeling)

In [ ]:
# # # Train YOLOv5s on COCO128 for 3 epochs
# %cd /content/yolov5
# !python train.py --img 544 --batch 32 --epochs 800 --data '/content/drive/MyDrive/droplet_classification/data_files/data.yaml' --weights '' --cfg ./models/yolov5m.yaml --cache

In [ ]:
# %cp /content/yolov5/runs/train/exp/weights/best.pt /content/drive/MyDrive/droplet_classification/data_files/yolov5_weights_365_best.pt

# 2. Test: Table FPS

mAP calculation for droplet model test set

In [ ]:
# Run YOLOv5s on COCO test-dev2017 using --task test
%cd /content/yolov5
!python val.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov5_weights_365_best.pt --img 544 --data /content/drive/MyDrive/droplet_classification/data_files/data.yaml --task test

/content/yolov5
val: data=/content/drive/MyDrive/droplet_classification/data_files/data.yaml, weights=['/content/drive/MyDrive/droplet_classification/data_files/yolov5_weights_365_best.pt'], batch_size=32, imgsz=544, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-187-gf3085ac torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 290 layers, 20865057 parameters, 0 gradients, 48.0 GFLOPs
test: Scanning '../test/labels' images and labels...70 found, 0 missing, 0 empty, 0 corrupted: 100% 70/70 [00:00<00:00, 876.39it/s]
test: New cache created: ../test/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:05<00:00,  1.69s/it]
                 all         70       1653      0.941     

# 3. Conduct average run times across test set: Table FPS

In [ ]:
# %cd /content/yolov5
# !python detect.py --weights /content/drive/MyDrive/droplet_classification/data_files/yolov5_weights_365_best.pt --img 544 --conf-thres 0.6 --source=../test/images

# Computer Statistics

In [ ]:
# !nvidia-smi -L
# !nvidia-smi

In [ ]:
# !lscpu |grep 'Model name'

# #no.of sockets i.e available slots for physical processors
# !lscpu | grep 'Socket(s):'

# #no.of cores each processor is having 
# !lscpu | grep 'Core(s) per socket:'

# #no.of threads each core is having
# !lscpu | grep 'Thread(s) per core'

# !lscpu | grep "L3 cache" 

# #if it had turbo boost it would've shown Min and Max MHz also but it is only showing current frequency this means it always operates at shown frequency
# !lscpu | grep "MHz"

# #memory that we can use
# !free -h --si | awk  '/Mem:/{print $2}'

# #hard disk space that we can use
# !df -h / | awk '{print $4}'